In [8]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data[:, :]  
y = iris.target
print(iris.keys())
print(iris.target_names)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
['setosa' 'versicolor' 'virginica']


# KMeans
- K 평균 알고리즘
  - 알고리즘이 찾을 클러스터 수 k개 지정
  - 센트로이드 랜덤 초기화
  - 샘플에 레이블 할당, 센트로이드 업데이트 -> 센트로이드 변동 없을 때까지 반복
  - 제한된 횟수이내 수렴 보장
- 하드 군집
    - 샘플당 클러스터 하나 부여
    - 클러스터의 크기가 다르면 잘 작동하지 않는다.
- 소프트 군집
    - 샘플당 클러스터들의 점수 부여
    - 점수: 샘플과 센트로이드 간 거리, 샘플과 센트로이드의 유사점수(친화성 점수)
- 한계
    - k 지정
    - 최적답을 찾기 위해 여러번 수행해야함
    - 클러스터간 크기, 밀집도, 원형 분포가 아닌 경우 잘 작동 안 함
        - 특성 스케일 중요한 이유

In [9]:
from sklearn.cluster import KMeans

k=3
kmeans = KMeans(n_clusters=k)
y_pred= kmeans.fit_predict(X)

/opt/homebrew/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
print(y_pred is kmeans.labels_)

True


In [13]:
print(kmeans.cluster_centers_)

[[5.006      3.428      1.462      0.246     ]
 [5.9016129  2.7483871  4.39354839 1.43387097]
 [6.85       3.07368421 5.74210526 2.07105263]]


### 이너셔
- 성능지표
- 샘플과 가장 가까운 센트로이드의 제곱 거리 합
- score는 이니셔의 음수값

In [14]:
kmeans.inertia_

78.85144142614601

In [15]:
kmeans.score(X)

-78.85144142614601

### KMeans++ 알고리즘
- 센트로이드와 거리가 먼 센트로이트 선택하는 똑똑한 초기화

### 속도개선
- 삼각부등식으로 계산량 줄이기
    - AC <= AB+BC
- 샘플과 센트로이드 거리 상하한선 유지

### 미니 배치 KMeans
- 센트로이드를 조금씩 업데이트
- 메모리 절약, 속도 매우 개선
    - 방법1 PCA memmap (??
    - 방법2 partial_fit
- 단점
    - 초기화, 좋은 미니배치 선택 직접해야함
    - 이니셔가 일반 kmeans보다 나쁨
        - k가 증가할 수록 이니셔가 줄어드는 경향이 있기 때문에 성능저하 비율이 커짐=

# 최적의 K 찾기
- 후보 1 이니셔
    - k가 커질 수록 작아지는 값이라 기각
- 후보 2 엘보
    - 이니셔를 k에 대한 함수로 나타내면 k'까지는 빠르게 감소하고 그 이후 느리게 감소함
- 후보 3 실루엣 점수
    - 각 샘플의 실루엣 계수 평균
    - 샘플의 실루엣 계수 = (b-a)/max(b-a)
        - a: 이 샘플과 같은 클러스터 내 다른 샘플들과의 평균 거리
        - b: 가장 가까운 클러스터의 샘플까지의 평균 거리
        - 값의 범위 -1 ~ +1
            - +1에 가까우면 잘 분류되었다.
            - 0이면 경계
            - -1에 가까우면 잘못 분류되었다.

In [29]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples

silhouette_score(X, kmeans.labels_)

0.5528190123564095

# 실루엣 다이어그램을 그려보자.
- 높이
    - 클러스터가 포함하는 샘플 수 
- 너비
    - 클러스터의 샘플들 실루엣 계수 넓을 수록 좋다.
- 엘보상 k=4, 실루엣 점수 상 k=3가 좋았지만 실루엣 계수 분포를 보니 k=6이 적당할 것 같다